# LangChain과 Zero-shot 프롬프트를 사용하여 베드락 모델 호출

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

이 노트북에서는 LLM을 사용하여 지원 엔지니어로부터 받은 고객 서비스 품질에 대해 부정적인 피드백을 제공한 고객에게 이메일 응답을 생성하는 방법을 보여드립니다. 

이 예에서는 Bedrock에서 제공하는 Amazon Titan 모델을 사용하겠습니다. 여기서는 [LangChain](https://python.langchain.com/docs/get_started/introduction.html)과 연동된 베드락 버전을 사용합니다. LangChain은 언어 모델 기반의 애플리케이션을 개발하기 위한 프레임워크입니다. 이 프레임워크의 핵심은 다양한 컴포넌트를 연결하여 대규모 언어 모델을 보강하고 고급 사용 사례를 만들 수 있다는 것입니다.

이 노트북에서는 LangChain에서 제공하는 Bedrock API를 사용하겠습니다. 이 예제에서 사용되는 프롬프트는 프롬프트 이외의 추가 컨텍스트를 제공하지 않기 때문에 제로 샷 프롬프트라고 합니다.

**참고:** *이 노트북은 AWS 환경 내부 또는 외부에서 실행할 수 있습니다*.

#### Context
이전 예제 `00_generate_w_bedrock.ipynb`에서는 Boto3 클라이언트를 사용하여 Amazon Bedrock API와 통신하는 방법을 살펴봤습니다. 이 노트북에서는 유사한 사용 사례에 대해 LangChain 프레임워크를 활용하기 위해 조금 더 복잡성을 추가해 보겠습니다. LangChain 프레임워크 내에서 Amazon Bedrock 통합을 사용하는 방법과 `PromptTemplate`을 사용하여 텍스트를 생성하는 방법을 살펴 보겠습니다.

#### Pattern
저희는 추가 예제를 제공하지 않고, 작업, 명령어 및 내부 모델에 대한 입력으로 구성된 입력을 Amazon Bedrock API의 LangChain 구현에 제공하여 출력을 생성하도록 할 것입니다. 이 글의 목적은 강력한 LLM이 어떻게 당면한 작업을 쉽게 이해하고 매력적인 출력을 생성하는지 보여드리기 위함입니다.

![](./images/bedrock_langchain.jpg)

#### Use case
Amazon Bedrock에서 모델의 생성 기능을 보여주기 위해 이메일 생성의 사용 사례를 살펴보겠습니다.

#### Persona
당신은 애니컴퍼니의 고객 서비스 매니저인 밥이고, 일부 고객이 고객 서비스에 만족하지 못하고 고객 지원 엔지니어가 제공하는 서비스에 대해 부정적인 피드백을 제공하고 있습니다. 이제 이러한 고객들에게 서비스 불만족에 대해 겸손하게 사과하고 신뢰를 회복하고 싶습니다. 이전 이메일 서신에서 얻은 고객의 정서에 맞춰 인간적이고 개인화된 이메일을 대량으로 생성하려면 LLM의 도움이 필요합니다.

#### Implementation
이 사용 사례를 충족하기 위해 이 노트북에서는 고객의 이전 이메일을 기반으로 감사 메모가 포함된 이메일을 생성하는 방법을 보여드리겠습니다. 여기서는 Amazon Bedrock LangChain 통합을 사용하는 Amazon Titan Text Large 모델을 사용하겠습니다.

## Setup

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 필요한 라이브러리가 설치되어 있는지 확인하고 베드락에 연결해야 합니다.

설정 작동 방식과 ⚠️ **변경이 필요한지 여부**에 대한 자세한 내용은 [Bedrock boto3 설정 노트북](../00_Intro/bedrock_boto3_setup.ipynb) 노트북을 참조하세요.

In [6]:
! pip list | grep langchain
! pip list | grep opensearch

langchain                            0.0.249
opensearch-py                        2.3.2


In [7]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import print_ww

In [8]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


## LangChain Integration을 사용하여 베드락 클라이언트 호출하기

llms에서 Bedrock 클래스의 인스턴스를 생성하는 것으로 시작하겠습니다. 여기에는 Amazon Bedrock에서 사용할 수 있는 모델의 `model_id`가 필요합니다.

선택적으로 이전에 생성한 boto3 `client`와 `temperature`, `topP`, `maxTokenCount` 또는 `stopSequences`와 같은 매개 변수를 보유할 수 있는 일부 `model_kwargs`를 전달할 수 있습니다.(매개 변수에 대한 자세한 내용은 Amazon Bedrock 콘솔에서 탐색할 수 있습니다)

Amazon Bedrock에서 사용 가능한 텍스트 생성 모델의 ID는 다음과 같습니다:

- amazon.titan-tg1-large
- ai21.j2-grande-instruct
- ai21.j2-jumbo-instruct
- anthropic.claude-instant-v1
- anthropic.claude-v1

모델마다 서로 다른 `model_kwargs`를 지원한다는 점에 유의하세요.

In [9]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {
    "maxTokenCount": 4096,
    "temperature": 0.5,
    "topP": 1,
    "stopSequences": ["||"],
}

textgen_llm = Bedrock(
    model_id="amazon.titan-tg1-large",
    client=boto3_bedrock,
    model_kwargs=inference_modifier,
)

LangChain에 'client'를 전달하면 라이브러리가 앞서 구성을 확인한 것과 동일한 boto3 클라이언트를 사용하도록 할 수 있습니다:

In [10]:
print(boto3_bedrock)
print(textgen_llm.client)

LangChain은 아마존 베드락 API를 추상화하여 사용 사례를 쉽게 구축할 수 있도록 했습니다. 프롬프트를 전달하면 적절한 API로 자동 라우팅되어 응답을 생성합니다. 응답 본문에서 결과를 추출할 필요 없이 텍스트 출력을 그대로 가져오기만 하면 됩니다.

고객 서비스 관리자가 고객에게 보낼 이메일을 생성하는 프롬프트를 준비해 보겠습니다.

In [11]:
response = textgen_llm("""Write an email from Bob, Customer Service Manager, 
to the customer "John Doe" that provided negative feedback on the service 
provided by our customer support engineer.||:""")

print_ww(response)


Dear John Doe,

We are sorry that you had a bad experience with our customer service and that the engineer you dealt
with did not meet your expectations.

Please be assured that we take feedback from our customers very seriously and we will investigate
the matter internally to ensure that this does not happen again.

We would appreciate it if you could provide us with more details about the incident so that we can
take appropriate action.

Thank you for bringing this to our attention and we apologize again for any inconvenience caused.

Best regards,
Bob Customer Service Manager


## Conclusion
이제 아마존 베드락 API에 추상화 계층을 제공하는 `LangChain` 프레임워크 사용을 실험해 보았습니다. 이 프레임워크를 사용하여 고객의 부정적인 피드백으로 인해 고객에게 응답하는 이메일을 생성하는 사용 사례를 보았습니다.

### Take aways
- 이 노트북을 수정하여 Anthropic Claude 및 AI21 Labs Jurassic 모델과 같은 Amazon Bedrock을 통해 제공되는 다양한 모델을 실험해 보세요.
- 프롬프트를 특정 사용 사례에 맞게 변경하고 다양한 모델의 출력을 평가하세요.
- 다양한 매개 변수를 사용해 서비스의 지연 시간과 응답성을 파악하세요.
- 다양한 프롬프트 엔지니어링 원칙을 적용하여 더 나은 결과를 얻으세요.

## Thank You